# Expected Passes Model

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os, sys
# We have to append the parent folder because we are running this from inside notebook
sys.path.append("../")
import pystatsbomb as sb
from pystatsbomb.plotting import plotpitch, plotpitch_bokeh
from pystatsbomb.plotting import pass_rose, getArrow
from pystatsbomb.passingmodel import PassingModel
import datetime
import pandas as pd
import pickle
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models import Arrow, OpenHead, Label
from bokeh.io import show, output_notebook
pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline

In [3]:
output_notebook()

Loading BokehJS ...

# Table of Contents

# Download World Cup Statsbomb Data

In [4]:
c = sb.Client(local_source_dir=None)
c.get_competitions()
# Check to see what competitions are returned
print([{comp["competition_name"]: comp["competition_id"]}  for comp in c.competitions])

comp_id = 43  # FIFA World Cup
match_ids = None  # Get all data
deep_lineup_parse = True

c.get_all_sb_data(
    comp_id=comp_id,
    match_ids=match_ids,
    deep_lineup_parse=deep_lineup_parse,
    toPandas=True)

df_matches = c.df_matches.set_index('match_id')
df_lineups = c.df_lineups
df_events = c.df_events

df_deep_lineups = c.df_deep_lineup
df_play_time = c.df_play_time
df_positions = c.df_positions

[{"FA Women's Super League": 37}, {'FIFA World Cup': 43}, {'NWSL': 49}]
Beginning Deep Lineup Parse


In [5]:
# Define World Cup Team Results
results = {
    'Argentina': 'Round of 16', 'Australia': 'Group Stage', 'Belgium': 'Semi-finals',
    'Brazil': 'Quarter-finals', 'Colombia': 'Round of 16', 'Costa Rica': 'Group Stage',
    'Croatia': 'Final', 'Denmark': 'Round of 16', 'Egypt': 'Group Stage',
    'England': 'Semi-finals', 'France': 'Final', 'Germany': 'Group Stage',
    'Iceland': 'Group Stage', 'Iran': 'Group Stage', 'Japan': 'Round of 16',
    'Mexico': 'Round of 16', 'Morocco': 'Group Stage', 'Nigeria': 'Group Stage',
    'Panama': 'Group Stage', 'Peru': 'Group Stage', 'Poland': 'Group Stage',
    'Portugal': 'Round of 16', 'Russia': 'Quarter-finals', 'Saudi Arabia': 'Group Stage',
    'Senegal': 'Group Stage', 'Serbia': 'Group Stage', 'South Korea': 'Group Stage',
    'Spain': 'Round of 16', 'Sweden': 'Quarter-finals', 'Switzerland': 'Round of 16',
    'Tunisia': 'Group Stage', 'Uruguay': 'Quarter-finals'}

# Run the Passing Model

In [6]:
# Initialize the Passing model with events and positions
passmodel = PassingModel(df_events, df_positions)

In [7]:
box_grid_model_player = passmodel.runPassModel(pass_model_type="box_grid", agg_level="player") 
box_grid_model_team = passmodel.runPassModel(pass_model_type="box_grid", agg_level="team") 

Pass 0 of 57975: 0.0% 
Pass 5000 of 57975: 8.62% 
Pass 10000 of 57975: 17.25% 
Pass 15000 of 57975: 25.87% 
Pass 20000 of 57975: 34.5% 
Pass 25000 of 57975: 43.12% 
Pass 30000 of 57975: 51.75% 
Pass 35000 of 57975: 60.37% 
Pass 40000 of 57975: 69.0% 
Pass 45000 of 57975: 77.62% 
Pass 50000 of 57975: 86.24% 
Pass 55000 of 57975: 94.87% 
Pass 0 of 57975: 0.0% 
Pass 5000 of 57975: 8.62% 
Pass 10000 of 57975: 17.25% 
Pass 15000 of 57975: 25.87% 
Pass 20000 of 57975: 34.5% 
Pass 25000 of 57975: 43.12% 
Pass 30000 of 57975: 51.75% 
Pass 35000 of 57975: 60.37% 
Pass 40000 of 57975: 69.0% 
Pass 45000 of 57975: 77.62% 
Pass 50000 of 57975: 86.24% 
Pass 55000 of 57975: 94.87% 


In [8]:
#box_grid_model_player.head()

In [9]:
box_grid_model_team["Round"] = box_grid_model_team["Team"].map(results)

In [10]:
import yaml
from bokeh.themes import Theme
from bokeh.models.widgets import Tabs
# os methods for manipulating paths
from os.path import dirname, join
from pystatsbomb.passmodelviz import team_tab, player_tab

In [11]:
def modify_doc(doc):

    player = box_grid_model_player
    team = box_grid_model_team
    
    tab1 = player_tab(player)
    tab2 = team_tab(team)

    tabs = Tabs(tabs = [tab1, tab2])
    doc.add_root(tabs)

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                toolbar_location: above
                height: 600
                width: 900
    """))


In [12]:
show(modify_doc)